In [0]:
!pip install dspy-ai openai mlflow pandas numpy

In [0]:
dbutils.library.restartPython()

In [0]:
# ============================================================
# セル2: ライブラリインポート
# ============================================================

import os
import json
from typing import List, Dict
from openai import OpenAI
import mlflow
import pandas as pd
from datetime import datetime

client = OpenAI()
print("✓ 環境セットアップ完了")

In [0]:
# ============================================================
# セル3: テストデータセット作成（修正版）
# ============================================================

# 顧客サマリー生成のテストケース
test_customers = [
    {
        "customer_id": "C001",  # idからcustomer_idに変更
        "name": "山田太郎",
        "age": 35,
        "occupation": "会社員",
        "purchase_history": "過去6ヶ月で電化製品を3回購入",
        "inquiries": "配送に関する問い合わせ2件、製品の使い方1件",
        "notes": "メールマガジン購読中、次回購入時10%割引クーポン保有"
    },
    {
        "customer_id": "C002",
        "name": "佐藤花子",
        "age": 28,
        "occupation": "フリーランスデザイナー",
        "purchase_history": "初回購入、ノートPC 1台",
        "inquiries": "製品仕様の詳細確認、納期確認",
        "notes": "企業向けアカウント検討中、見積依頼あり"
    },
    {
        "customer_id": "C003",
        "name": "鈴木一郎",
        "age": 52,
        "occupation": "自営業",
        "purchase_history": "月1回ペースで事務用品を購入、累計15回",
        "inquiries": "請求書発行依頼が多い",
        "notes": "法人契約希望、大量発注の可能性あり"
    }
]

print(f"✓ テストデータ: {len(test_customers)}件")


In [0]:
# ============================================================
# セル4: プロンプトバージョン定義
# ============================================================

# 評価対象の複数プロンプトバージョン
prompt_versions = {
    "v1_basic": """
以下の顧客情報を200文字以内で要約してください。

顧客ID: {customer_id}
名前: {name}
年齢: {age}
職業: {occupation}
購入履歴: {purchase_history}
問い合わせ履歴: {inquiries}
備考: {notes}
""",
    
    "v2_structured": """
以下の顧客情報を、重要なポイントを押さえて簡潔に要約してください。

【基本情報】
- 顧客ID: {customer_id}
- 名前: {name}（{age}歳、{occupation}）

【取引状況】
- 購入: {purchase_history}
- 問い合わせ: {inquiries}

【特記事項】
{notes}

【要約要件】
- 200文字以内
- 営業担当者が次のアクションを判断できる内容
- 数値・具体的事実を優先
""",
    
    "v3_actionable": """
営業担当者向けに、以下の顧客情報から次のアクション判断に必要な要点を抽出してください。

顧客: {name}（ID: {customer_id}、{age}歳、{occupation}）
購入履歴: {purchase_history}
問い合わせ: {inquiries}
備考: {notes}

以下の観点で200文字以内で要約:
1. 顧客の購買パターン
2. 現在のニーズや課題
3. 次に提案すべき内容
"""
}

print(f"✓ プロンプトバージョン: {len(prompt_versions)}個")
for version in prompt_versions.keys():
    print(f"  - {version}")


In [0]:
# ============================================================
# セル5: 推論実行関数
# ============================================================

def generate_summary(customer: Dict, prompt_template: str, model: str = "gpt-4o-mini") -> str:
    """
    プロンプトテンプレートに顧客データを埋め込んで要約生成
    """
    prompt = prompt_template.format(**customer)
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "あなたは優秀なカスタマーサポート担当者です。"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=300
    )
    
    return response.choices[0].message.content

print("✓ 推論関数定義完了")


In [0]:
# ============================================================
# セル6: 評価関数（LLM-as-a-Judge）
# ============================================================

def evaluate_summary(customer: Dict, summary: str) -> Dict:
    """
    生成された要約を評価
    
    Returns:
        {
            "score": float (0.0-1.0),
            "reasoning": str,
            "metrics": {
                "conciseness": float,
                "clarity": float,
                "actionability": float
            }
        }
    """
    
    evaluation_prompt = f"""
以下の顧客要約を評価してください。

【元データ】
顧客: {customer['name']} ({customer['age']}歳、{customer['occupation']})
購入履歴: {customer['purchase_history']}
問い合わせ: {customer['inquiries']}
備考: {customer['notes']}

【生成された要約】
{summary}

【評価基準】
1. 簡潔性 (0-10): 200文字以内で無駄がないか
2. 明瞭性 (0-10): 重要情報が明確に伝わるか
3. 実用性 (0-10): 次のアクション判断に役立つか

以下のJSON形式で評価を出力してください:
{{
  "conciseness": <0-10の整数>,
  "clarity": <0-10の整数>,
  "actionability": <0-10の整数>,
  "reasoning": "<評価理由>"
}}
"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "あなたは厳正な品質評価者です。"},
            {"role": "user", "content": evaluation_prompt}
        ],
        temperature=0.1,
        response_format={"type": "json_object"}
    )
    
    result = json.loads(response.choices[0].message.content)
    
    # 総合スコア計算（平均を0-1に正規化）
    total_score = (
        result["conciseness"] + 
        result["clarity"] + 
        result["actionability"]
    ) / 30.0
    
    return {
        "score": total_score,
        "reasoning": result.get("reasoning", ""),
        "metrics": {
            "conciseness": result["conciseness"] / 10.0,
            "clarity": result["clarity"] / 10.0,
            "actionability": result["actionability"] / 10.0
        }
    }

print("✓ 評価関数定義完了")


In [0]:
# ============================================================
# セル7: 評価実験実行
# ============================================================

# MLflow実験設定
mlflow.set_experiment(experiment_id=2762244084694129)

results = []

print("=" * 60)
print("プロンプト評価実験開始")
print("=" * 60)

for version_name, prompt_template in prompt_versions.items():
    print(f"\n📋 評価中: {version_name}")
    
    with mlflow.start_run(run_name=version_name):
        # プロンプトをログ
        mlflow.log_param("プロンプトバージョン", version_name)
        mlflow.log_text(prompt_template, "プロンプトテンプレート.txt")
        
        version_scores = []
        
        for customer in test_customers:
            print(f"  → {customer['name']}")
            
            # 要約生成
            summary = generate_summary(customer, prompt_template)
            
            # 評価実行
            evaluation = evaluate_summary(customer, summary)
            
            # 結果記録
            result = {
                "プロンプトバージョン": version_name,
                "顧客ID": customer["customer_id"],
                "顧客名": customer["name"],
                "要約": summary,
                "総合スコア": evaluation["score"],
                "簡潔性": evaluation["metrics"]["conciseness"],
                "明瞭性": evaluation["metrics"]["clarity"],
                "実用性": evaluation["metrics"]["actionability"],
                "評価理由": evaluation["reasoning"]
            }
            
            results.append(result)
            version_scores.append(evaluation["score"])
            
            print(f"     スコア: {evaluation['score']:.2%}")
        
        # バージョンごとの平均スコアをログ
        avg_score = sum(version_scores) / len(version_scores)
        mlflow.log_metric("平均スコア", avg_score)
        mlflow.log_metric("平均_簡潔性", sum(r["簡潔性"] for r in results[-len(test_customers):]) / len(test_customers))
        mlflow.log_metric("平均_明瞭性", sum(r["明瞭性"] for r in results[-len(test_customers):]) / len(test_customers))
        mlflow.log_metric("平均_実用性", sum(r["実用性"] for r in results[-len(test_customers):]) / len(test_customers))
        
        print(f"  ✓ 平均スコア: {avg_score:.2%}")

print("\n" + "=" * 60)
print("✓ 評価実験完了")
print("=" * 60)